# Q-Learning

In [1]:
import numpy as np
import pandas as pd
import random

# TODO: DEFINE ACTION SPACE FOR EACH STATE

### Define States ###
states = []
for i in range(14):
    for j in range(20):
        states.append((i+1, j+1))

# Left wall states
left_wall = []
for i in range(16):
    left_wall.append((i, 0))
# Upper wall states
upper_wall = []
for j in range(22):
    upper_wall.append((15, j))
# Right wall
right_wall = []
for i in range(16):
    left_wall.append((i, 21))
# Bottom wall
bottom_wall = []
for j in range(22):
    upper_wall.append((0, j))

# Concat to walls
walls = left_wall + upper_wall + right_wall + bottom_wall

# Vertical wall + tree (right from Trainer)
for i in range(4, 14+1):
	walls.append((i, 4))

# Horizontal trees + building + mountain
for i in range(9, 16):
    for j in range(4, 20+1):
	    walls.append((i, j))
walls.remove((9,8)) # Do not add door to walls

# Horizontal lower wall 
for j in range(4, 16+1):
	walls.append((4, j))
walls.remove((4,8)) # Do not add staircase to walls

# Vertial Call right from charmander
for i in range(3, 9+1):
	walls.append((i, 19)) 
    
# Vertical Wall left from charmander
walls.append((8, 12))
walls.append((7, 12))
for j in range(12, 20):
    walls.append((6, j))
walls.remove((6, 16))


### Define Rewards ###
rewards = {}
for state in states:
    # Catch Bisasam
    #if state == (8, 10):
        #rewards[state] = 100
    # Catch Schiggy
    #elif state == (2, 13):
        #rewards[state] = 100
    # Catch Charmander
    if state == (8, 18):
        rewards[state] = 100
    # Go to building
    #elif state == (9, 8):
        #rewards[state] = 1
    # For all other States
    else:
        rewards[state] = -0.5
    
# Function that returns the valid next state, given current state and action
def getNextState(a: str, s: tuple) -> tuple:
    if a == "U":
        next_state = (s[0] - 1, s[1])
        if next_state in walls:
            next_state = (s[0], s[1])

    if a == "D":
        next_state = (s[0] + 1, s[1])
        if next_state in walls:
            next_state = (s[0], s[1])    

    if a == "L":
        next_state = (s[0], s[1] - 1)
        if next_state in walls:
            next_state = (s[0], s[1])

    if a == "R":
        next_state = (s[0], s[1] + 1)
        if next_state in walls:
            next_state = (s[0], s[1])
    
    return next_state


### Define Actions ###
actions = {}
for s in states:
    temp_action = []

    if getNextState("U", s) != s:
        temp_action.append("U")

    if getNextState("D", s) != s:
        temp_action.append("D")

    if getNextState("L", s) != s:
        temp_action.append("L")

    if getNextState("R", s) != s:
        temp_action.append("R")

    actions[s] = temp_action


################################################
### Define Q-Table and initialize it with 0  ###
################################################
n_states = len(states)      
n_action = 4

# Initialize Q_Table to all Zeros and convert to pd DataFrame
Q = np.zeros(shape=(n_states, n_action))
Q = pd.DataFrame(Q, columns=["U", "D", "L", "R"], index=states)

print("Q-Table at the beginning:")
display(Q)

### Define Hyperparams ###
GAMMA = 0.9
EPSILON = 0.95
ALPHA = 0.9
EPISODE = 1000

# Q-Learning algorithm:
for i in range(EPISODE):
    reward_history = []

    # Init (a, b) to (1, 4) so that the while loop always triggers and shuffles new
    (a, b) = (0, 0)

    # Get random start state
    while (a, b) in walls:
        a = np.random.choice(a=np.arange(15))
        b = np.random.choice(a=np.arange(21))

    state = (a, b)
    print(f"Start State at Epsiode {i} is {state}")
    
    # Simulate Episode
    for i in range(100):
        #print(f"Current state: {state}")
        
        # Get next state randomly (Exploration)
        # Epsilon decreases in relation to the current episode
        # Thus Epsilon is high at the beginning and low at the end
        if random.random() < EPSILON**i**(1/3):
            # Get random action
            a = np.random.choice([action for action in actions[state]])
            next_state = getNextState(a, state)
            #print(f"Random Action '{a}' has been taken. Resulting next state is {next_state}")
        # Get next state from Q-Table (Exploitation)
        else:
            valid_actions = [action for action in actions[state]]
            #print(f"Valid Action in state {state} is: {valid_actions}")
            # Get max_a q(state, a) from the Q-Table
            a = list(Q.loc[[state], [action for action in actions[state]]].idxmax(axis=1).values)[0]
            #print(f"Max Action in state {state} is: {a}")
            next_state = getNextState(a, state)
            #print(f"Next state is: {next_state}")

        # Update Q-Table
        Q.at[state, a] = Q.at[state, a] + ALPHA * (rewards[next_state] + GAMMA * float(Q.loc[[next_state], :].max(axis=1)) - Q.at[state, a])
        reward_history.append(rewards[next_state])
        
        # If we reach one terminal state: end the episode
        #if next_state == (9, 8) or next_state == (2, 13) or next_state == (8, 10) or next_state == (8, 18):
            #rewards[next_state] = -0.5
            #break
            
        if next_state == (8, 18):
            break

        state = next_state
        

print("Training is finished!")
print("Q-Table in the end:")
display(Q)


Q-Table at the beginning:


,U,D,L,R
"(1, 1)",0.0,0.0,0.0,0.0
"(1, 2)",0.0,0.0,0.0,0.0
"(1, 3)",0.0,0.0,0.0,0.0
"(1, 4)",0.0,0.0,0.0,0.0
"(1, 5)",0.0,0.0,0.0,0.0
...,...,...,...,...
"(14, 16)",0.0,0.0,0.0,0.0
"(14, 17)",0.0,0.0,0.0,0.0
"(14, 18)",0.0,0.0,0.0,0.0
"(14, 19)",0.0,0.0,0.0,0.0


Start State at Epsiode 0 is (11, 2)
Start State at Epsiode 1 is (13, 1)
Start State at Epsiode 2 is (4, 17)
Start State at Epsiode 3 is (8, 20)
Start State at Epsiode 4 is (3, 17)
Start State at Epsiode 5 is (5, 14)
Start State at Epsiode 6 is (7, 3)
Start State at Epsiode 7 is (5, 7)
Start State at Epsiode 8 is (8, 8)
Start State at Epsiode 9 is (1, 15)
Start State at Epsiode 10 is (5, 13)
Start State at Epsiode 11 is (6, 1)
Start State at Epsiode 12 is (2, 10)
Start State at Epsiode 13 is (8, 16)
Start State at Epsiode 14 is (8, 9)
Start State at Epsiode 15 is (1, 3)
Start State at Epsiode 16 is (8, 2)
Start State at Epsiode 17 is (2, 9)
Start State at Epsiode 18 is (8, 17)
Start State at Epsiode 19 is (14, 3)
Start State at Epsiode 20 is (7, 10)
Start State at Epsiode 21 is (8, 10)
Start State at Epsiode 22 is (4, 17)
Start State at Epsiode 23 is (6, 8)
Start State at Epsiode 24 is (6, 10)
Start State at Epsiode 25 is (8, 20)
Start State at Epsiode 26 is (8, 3)
Start State at Epsiod

,U,D,L,R
"(1, 1)",0.0,32.003517,0.000000,32.333403
"(1, 2)",0.0,36.843114,28.303327,36.441937
"(1, 3)",0.0,41.531577,32.268339,41.021950
"(1, 4)",0.0,46.881866,36.115218,46.639571
"(1, 5)",0.0,52.890220,41.468056,52.652828
...,...,...,...,...
"(14, 16)",0.0,0.000000,0.000000,0.000000
"(14, 17)",0.0,0.000000,0.000000,0.000000
"(14, 18)",0.0,0.000000,0.000000,0.000000
"(14, 19)",0.0,0.000000,0.000000,0.000000


# Print optimal Policy 🚀


In [2]:
# Print optimal policy 
for state in states:
    # Skip terminal states and wall
    if state in walls:
        pass
    # All other non-terminal states 
    else:
        best_action = list(Q.loc[[state], :].idxmax(axis=1).values)[0]
        print(f"The best action in state {state} is {best_action}")

print("############################")

# Recursive function to draw path beginning at s (should be s = (14, 2))
def printPath(s):
    if s == (8, 18):
        print(s)
        print("Terminal state has been reached.")
        return
    #else:   
    print(s)
    best_action = str(list(Q.loc[[s], :].idxmax(axis=1).values)[0])
    next_state = getNextState(best_action, s)
    printPath(next_state)

printPath((14, 2))

print(f"Total Reward collected {sum(reward_history)}")


The best action in state (1, 1) is R
The best action in state (1, 2) is D
The best action in state (1, 3) is D
The best action in state (1, 4) is D
The best action in state (1, 5) is D
The best action in state (1, 6) is D
The best action in state (1, 7) is R
The best action in state (1, 8) is D
The best action in state (1, 9) is L
The best action in state (1, 10) is R
The best action in state (1, 11) is D
The best action in state (1, 12) is D
The best action in state (1, 13) is R
The best action in state (1, 14) is R
The best action in state (1, 15) is R
The best action in state (1, 16) is D
The best action in state (1, 17) is D
The best action in state (1, 18) is L
The best action in state (1, 19) is L
The best action in state (1, 20) is L
The best action in state (2, 1) is R
The best action in state (2, 2) is R
The best action in state (2, 3) is D
The best action in state (2, 4) is R
The best action in state (2, 5) is D
The best action in state (2, 6) is R
The best action in state (2

In [3]:

print(f"Total Reward collected {sum(reward_history)}")

Total Reward collected 56.5
